# Model Evaluation and Hyperparameter Tuning for Pediatric Appendicitis Diagnosis (Part 2)

This notebook continues our model evaluation, focusing on:
- Advanced evaluation metrics for medical diagnosis
- Model interpretation and feature importance
- Threshold optimization for clinical decision-making
- Model calibration

In [ ]:
# Import necessary libraries (similar to Part 1)
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from pathlib import Path
import joblib
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, roc_curve, precision_recall_curve, average_precision_score,
    classification_report, log_loss, brier_score_loss
)
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
import shap
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Add project root to path
sys.path.append('..')

# Import project modules
from src.data_processing.preprocess import load_data, handle_missing_values, optimize_memory

# Set plot styling
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 14

## 1. Load the Data and Best Model from Part 1

First, let's load the data and the best tuned model from Part 1.

In [ ]:
print("Loading data and model...")

# Load synthetic data
data_path = '../DATA/synthetic_appendicitis_data.csv'
df = pd.read_csv(data_path)

# Prepare features and target
X = df[['Age', 'Temperature', 'WBC', 'CRP', 'Pain_Duration', 'Neutrophil_Percent']]
y = df['Appendicitis']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Load the scaler
try:
    scaler = joblib.load('../models/standard_scaler.pkl')
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
except:
    print("Scaler not found, creating a new one...")
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

# Try to load the best model (if it exists)
model_dir = Path('../models')
model_files = list(model_dir.glob('tuned_*_model.pkl')) if model_dir.exists() else []

if model_files:
    best_model_path = model_files[0]  # Take the first tuned model file
    best_model = joblib.load(best_model_path)
    best_model_name = best_model_path.stem.replace('tuned_', '').replace('_model', '').replace('_', ' ').title()
    print(f"Loaded model: {best_model_name} from {best_model_path}")
else:
    print("No tuned model found, please run Part 1 first or provide a model.")
    # For demonstration, we'll create a simple model
    from sklearn.ensemble import RandomForestClassifier
    best_model = RandomForestClassifier(random_state=42)
    best_model.fit(X_train_scaled, y_train)
    best_model_name = "Random Forest"

# Generate predictions
y_pred = best_model.predict(X_test_scaled)
y_prob = best_model.predict_proba(X_test_scaled)[:, 1]

## 2. Advanced Evaluation Metrics for Medical Diagnosis

In medical diagnosis, especially for conditions like appendicitis, we need to consider more specific metrics than just accuracy or AUC.

In [ ]:
# Calculate confusion matrix elements
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Calculate additional metrics for medical diagnosis
sensitivity = tp / (tp + fn)  # Same as Recall
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)  # Positive Predictive Value, same as Precision
npv = tn / (tn + fn)  # Negative Predictive Value
accuracy = (tp + tn) / (tp + tn + fp + fn)
f1 = 2 * (ppv * sensitivity) / (ppv + sensitivity)
diagnostic_odds_ratio = (tp * tn) / (fp * fn)
positive_likelihood_ratio = sensitivity / (1 - specificity)
negative_likelihood_ratio = (1 - sensitivity) / specificity

# Report clinical metrics
print("\nClinical Diagnostic Metrics:")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Positive Predictive Value (Precision): {ppv:.4f}")
print(f"Negative Predictive Value: {npv:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Diagnostic Odds Ratio: {diagnostic_odds_ratio:.4f}")
print(f"Positive Likelihood Ratio: {positive_likelihood_ratio:.4f}")
print(f"Negative Likelihood Ratio: {negative_likelihood_ratio:.4f}")

In [ ]:
# Visualize advanced metrics
metrics = ['Sensitivity', 'Specificity', 'PPV', 'NPV', 'Accuracy', 'F1']
values = [sensitivity, specificity, ppv, npv, accuracy, f1]

plt.figure(figsize=(12, 6))
bars = plt.bar(metrics, values, color=sns.color_palette('viridis', len(metrics)))
plt.title('Clinical Evaluation Metrics')
plt.ylabel('Score')
plt.ylim(0, 1.0)
plt.grid(axis='y', alpha=0.3)

# Add data labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.savefig('../figures/clinical_metrics.png')
plt.show()

## 3. Model Interpretation and Feature Importance

Understanding which features are most important for the model's predictions is crucial for clinical applicability.

In [ ]:
# Check if the model has feature_importances_ attribute
if hasattr(best_model, 'feature_importances_'):
    # For tree-based models
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': best_model.feature_importances_
    }).sort_values(by='Importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature', data=feature_importance, palette='viridis')
    plt.title(f'Feature Importance for {best_model_name}')
    plt.tight_layout()
    plt.savefig('../figures/feature_importance.png')
    plt.show()
    
elif hasattr(best_model, 'coef_'):
    # For linear models
    coefficients = pd.DataFrame({
        'Feature': X.columns,
        'Coefficient': best_model.coef_[0] if len(best_model.coef_.shape) > 1 else best_model.coef_
    }).sort_values(by='Coefficient', key=abs, ascending=False)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Coefficient', y='Feature', data=coefficients, palette='viridis')
    plt.title(f'Feature Coefficients for {best_model_name}')
    plt.axvline(x=0, color='k', linestyle='--')
    plt.tight_layout()
    plt.savefig('../figures/feature_coefficients.png')
    plt.show()
    
else:
    # For models without direct feature importance, try permutation importance
    print("This model doesn't have built-in feature importance. Using SHAP values for interpretation.")

In [ ]:
# SHAP values for model interpretation
try:
    # Sample for SHAP analysis (using a subset for speed)
    X_sample = pd.DataFrame(X_test_scaled[:100], columns=X.columns)
    
    # Create explainer
    explainer = shap.Explainer(best_model, X_sample)
    shap_values = explainer(X_sample)
    
    # Summary plot
    plt.figure()
    shap.summary_plot(shap_values, X_sample, plot_type="bar", show=False)
    plt.title(f'SHAP Feature Importance for {best_model_name}')
    plt.tight_layout()
    plt.savefig('../figures/shap_importance.png')
    plt.show()
    
    # Detailed SHAP plots
    plt.figure()
    shap.summary_plot(shap_values, X_sample, show=False)
    plt.title(f'SHAP Summary Plot for {best_model_name}')
    plt.tight_layout()
    plt.savefig('../figures/shap_summary.png')
    plt.show()
except Exception as e:
    print(f"Could not generate SHAP plots due to: {e}")

## 4. Threshold Optimization for Clinical Decision-Making

Finding the optimal threshold for clinical decision-making based on cost-benefit considerations.

In [ ]:
# Calculate metrics at different thresholds
thresholds = np.linspace(0, 1, 101)
threshold_metrics = []

for threshold in thresholds:
    y_pred_th = (y_prob >= threshold).astype(int)
    
    # Skip if we have a division by zero issue
    if np.sum(y_pred_th) == 0 or np.sum(y_pred_th == 0) == 0:
        continue
    
    # Calculate metrics
    try:
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred_th).ravel()
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
        npv = tn / (tn + fn) if (tn + fn) > 0 else 0
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        f1 = 2 * (ppv * sensitivity) / (ppv + sensitivity) if (ppv + sensitivity) > 0 else 0
        
        threshold_metrics.append({
            'Threshold': threshold,
            'Sensitivity': sensitivity,
            'Specificity': specificity,
            'PPV': ppv,
            'NPV': npv,
            'Accuracy': accuracy,
            'F1': f1
        })
    except:
        continue

# Convert to DataFrame
threshold_df = pd.DataFrame(threshold_metrics)

In [ ]:
# Plot metrics vs threshold
plt.figure(figsize=(12, 8))
metrics_to_plot = ['Sensitivity', 'Specificity', 'PPV', 'NPV', 'Accuracy', 'F1']
for metric in metrics_to_plot:
    plt.plot(threshold_df['Threshold'], threshold_df[metric], label=metric, linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Metrics at Different Classification Thresholds')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../figures/threshold_metrics.png')
plt.show()

In [ ]:
# Find optimal thresholds for different metrics
optimal_thresholds = {
    'F1': threshold_df.loc[threshold_df['F1'].idxmax(), 'Threshold'],
    'Accuracy': threshold_df.loc[threshold_df['Accuracy'].idxmax(), 'Threshold'],
    # Youden's J statistic (Sensitivity + Specificity - 1)
    'Youden': threshold_df.loc[(threshold_df['Sensitivity'] + threshold_df['Specificity'] - 1).idxmax(), 'Threshold'],
    # Cost-benefit specific threshold for appendicitis
    # In medical context for appendicitis, missing a case (false negative) is typically considered worse than
    # a false positive, as untreated appendicitis can lead to serious complications
    # We could favor sensitivity over specificity with a 2:1 weight ratio
    'Clinical': threshold_df.loc[(2*threshold_df['Sensitivity'] + threshold_df['Specificity']).idxmax(), 'Threshold']
}

print("Optimal Thresholds:")
for metric, threshold in optimal_thresholds.items():
    print(f"{metric}: {threshold:.3f}")

# Highlight optimal thresholds on the plot
plt.figure(figsize=(12, 8))
for metric in metrics_to_plot:
    plt.plot(threshold_df['Threshold'], threshold_df[metric], label=metric, linewidth=2)

# Add vertical lines for optimal thresholds
colors = ['r', 'g', 'b', 'm']
for i, (metric, threshold) in enumerate(optimal_thresholds.items()):
    plt.axvline(x=threshold, color=colors[i], linestyle='--', label=f'{metric} Optimal: {threshold:.3f}')

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Optimal Classification Thresholds for Different Criteria')
plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.25), ncol=3)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../figures/optimal_thresholds.png')
plt.show()